In [26]:
import pandas as pd

url = 'https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=share_link' # orders.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orders = pd.read_csv(path)
orders = orders.loc[~orders.total_paid.isna(), :]
orders["created_date"] = pd.to_datetime(orders["created_date"])

In [27]:
url = 'https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=share_link' # orderlines.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orderlines = pd.read_csv(path)

In [28]:
url = 'https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=share_link' #products.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
products = pd.read_csv(path)
products.drop_duplicates(inplace=True)
products.loc[products.desc.isna(), 'desc'] = products.loc[products.desc.isna(), 'name']
products.loc[products.type.isna(), 'type'] = 'missing'
products = products.drop(columns='promo_price')

In [29]:
url = 'https://drive.google.com/file/d/1m1ThDDIYRTTii-rqM5SEQjJ8McidJskD/view?usp=share_link' #brands.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
brands = pd.read_csv(path)

### Orders

In [30]:
orders_trim = orders.loc[orders.order_id.isin(orderlines.id_order)]
orders_trim = orders_trim.loc[orders_trim.state == 'Completed']
orders_trim = orders_trim.loc[~(orders_trim.total_paid <= 0)]

### Products
1. Price and Promo Price should both be integers, but instead appear as text
2. In stock looks like Integer, but is thought out like Bool
3. What the what is Type supposed to mean?

In [31]:
# First toss out items with no Products price
no_price = products.loc[products.price.isna()]['price']
products = products.loc[~products.price.isin(no_price)].reset_index(drop=True)
# First get SKUs of products with bad prices, then get SKUs of those products which were actually ordered
# This way there's a sales price to compare to the Products price
# double_dot_skus = products.loc[products.price.str.contains('\d+\.\d+\.'), 'sku']
# ordered_double_dot_skus = orderlines.loc[orderlines.sku.isin(double_dot_skus)]
#Checking for different behavior with different numbers of digits before the first dot. Run one at a time.
# products.loc[(products.sku.isin(ordered_double_dot_skus)) & (products.price.str.contains('\d\d\d\.\d+\.'))]
# products.loc[(products.sku.isin(ordered_double_dot_skus)) & (products.price.str.contains('\d\d\.\d+\.'))]
# products.loc[(products.sku.isin(ordered_double_dot_skus)) & (products.price.str.contains('\d\.\d+\.'))]

In [32]:
# Case-based function to adjust prices
def prod_price_cleaner(price):
  if price.startswith('.', 1):
    price = price.replace('.','', 1)
    price = round(float(price) / 10, 2)
    return price
  if price.startswith('.',2):
    price = price[:5]
    price = float(''.join(price))
    return price
  if price.startswith('.', 3):
    price = price.replace('.','',1)
    price = round(float(price) / 100, 2)
    return price
  else:
    return round(float(price), 2)
    # return price

In [33]:
products['clean_price'] = products.price.apply(prod_price_cleaner)

In [34]:
products['clean_judge'] = products.clean_price.apply(lambda x: 'low' if x < 100 else 'high')
big = products.loc[products.clean_judge == 'high', 'sku']
little = products.loc[products.clean_judge == 'low', 'sku']

### Order lines
1. Unit price is a string
2. Does date need work?
3. There's an order of 999 items

In [35]:
orderlines["date"] = pd.to_datetime(orderlines["date"])

In [36]:
bad_batch = orderlines.loc[orderlines.unit_price.str.contains("\d+\.\d+\.\d+")].copy()

In [37]:
new = orderlines[~orderlines.unit_price.str.contains('\d+\.\d+\.\d+')].copy()
new['unit_price'] = pd.to_numeric(new['unit_price'])
high_low = new.groupby('sku').mean()['unit_price'].reset_index()
high_low

,sku,unit_price
0,8MO0001-A,12.960000
1,8MO0003-A,12.850000
2,8MO0007,21.369310
3,8MO0008,21.723333
4,8MO0009,17.842857
...,...,...
6829,ZAG0041,19.990000
6830,ZAG0042,19.990000
6831,ZEP0007,137.118000
6832,ZEP0008,104.120000


In [38]:
high_low2 = high_low.loc[high_low.sku.isin(bad_batch['sku'])]
high_low2['up_down'] = high_low2.unit_price.apply(lambda x: 'high' if x > 100 else 'low')
high2 = high_low2.loc[high_low2.up_down == 'high', 'sku']
low2 = high_low2.loc[high_low2.up_down == 'low', 'sku']
low2 = list(low2)
high2 = list(high2)

<ipython-input-38-5aa6ac4d7411>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_low2['up_down'] = high_low2.unit_price.apply(lambda x: 'high' if x > 100 else 'low')


In [39]:
def ol_price_cleaner2(price, sku):
  if sku in high2:
    price = price.replace('.','',1)
    return round(float(price), 2)
    # return price
  if sku in low2:
    price = price[-4:]
    return float(''.join(price))
    # return '.'.join(price)
  if sku in big:
    price = price.replace('.','',1)
    return round(float(price), 2)
  if sku in little:
    price = price.replace('.','',1)
    return round(float(price)/100, 2)
  else:
    return price

In [40]:
orderlines_trim = orderlines.loc[orderlines.id_order.isin(orders_trim.order_id)]
orderlines_trim['price_clean'] = orderlines_trim.loc[orderlines_trim.unit_price.str.contains('\d+\.\d+\.\d+')].apply(lambda x: ol_price_cleaner2(x['unit_price'], x['sku']), axis=1)
orderlines_trim['price_clean'] = pd.to_numeric(orderlines_trim['price_clean'], errors='coerce')
orderlines_trim['unit_price'] = orderlines_trim.apply(lambda x: x['price_clean'] if x['price_clean'] > 0 else x['unit_price'], axis=1)
orderlines_trim['unit_price'] = pd.to_numeric(orderlines_trim['unit_price'], errors='coerce')
orderlines_trim = orderlines_trim.loc[orderlines_trim.sku.isin(products.sku)]

<ipython-input-40-2f336a27f089>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orderlines_trim['price_clean'] = orderlines_trim.loc[orderlines_trim.unit_price.str.contains('\d+\.\d+\.\d+')].apply(lambda x: ol_price_cleaner2(x['unit_price'], x['sku']), axis=1)
<ipython-input-40-2f336a27f089>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orderlines_trim['price_clean'] = pd.to_numeric(orderlines_trim['price_clean'], errors='coerce')
<ipython-input-40-2f336a27f089>:4: SettingWithCopyWarning: 
A value is tr

,id,id_order,product_id,product_quantity,sku,unit_price,date,price_clean
7,1119116,299545,0,1,OWC0100,47.49,2017-01-01 01:46:16,NaN
8,1119119,299546,0,1,IOT0014,18.99,2017-01-01 01:50:34,NaN
9,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,NaN
11,1119126,299549,0,1,PAC0929,NaN,2017-01-01 02:07:42,NaN
18,1119134,299556,0,1,CRU0039-A,60.90,2017-01-01 02:20:14,NaN
...,...,...,...,...,...,...,...,...
293599,1649474,525664,0,1,TUC0207,16.52,2018-03-14 11:45:05,NaN
293616,1649512,527070,0,2,APP0698,9.99,2018-03-14 11:49:01,NaN
293621,1649522,527074,0,2,APP0698,9.99,2018-03-14 11:49:36,NaN
293644,1649565,527096,0,3,APP0698,9.99,2018-03-14 11:54:35,NaN


In [ ]:
orderlines_trim = orderlines_trim.loc[~orderlines_trim.unit_price.isna()]

In [ ]:
orderlines_trim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59833 entries, 7 to 293661
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                59833 non-null  int64         
 1   id_order          59833 non-null  int64         
 2   product_id        59833 non-null  int64         
 3   product_quantity  59833 non-null  int64         
 4   sku               59833 non-null  object        
 5   unit_price        59833 non-null  float64       
 6   date              59833 non-null  datetime64[ns]
 7   price_clean       1677 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 4.1+ MB


### Experiments

Short analysis on the two-dot prices:
Looks like there's two things going on. One is cheap prices where two have been shoved together.
2.976.59 being 2.97 and 6.59
The other is high prices where a "." comes in to indicate a thousands column.
1.120.99 being 1120.99 (which can be written 1,120.99 for readability)
Took some manual investigation, but I think we can make a function to sort cases based on average price and adjust appropriately.

Select for right-side price on lows.

In [ ]:
from collections import Counter
import re

In [ ]:
types = list(products.type.unique())

In [ ]:
products.loc[products.type == '8696', 'desc']
#docs and stands

In [ ]:
products.loc[products.type == '13855401', 'desc']
# keyboards

In [ ]:
type_dict = {}
for t in types:
  type_dict[t] = list(products.loc[products.type == t, 'desc'])
  # print(products.loc[products.type == t, ['desc', 'type']].head())

In [ ]:
type_words = {}
for td in type_dict.keys():
  string = ''
  for value in type_dict[td]:
    string += ' '+ value
    list = string.split()
    new_list = []
    for l in list:
      l = l.replace('.','').replace('+', '/').replace('/','').replace('your', '').replace('and','').replace('with','').replace('for','').replace('of','')
      new_list.append(l)
    counter = Counter(new_list)
    most_occur = counter.most_common(5)
    words = []
    for i in range(len(most_occur)):
      words.append(most_occur[i][0])
    type_words[td] = words
type_words

{'8696': ['', 'MacBook', 'support', 'Pro', 'aluminum'],
 '13855401': ['', 'Mac', 'Bluetooth', 'keyboard', 'Keyboard'],
 '1387': ['', 'Mac', 'mouse', 'Apple', 'Bluetooth'],
 '1230': ['', 'iPod', 'iPad', 'iPhone', 'Lightning'],
 '1364': ['RAM', 'Mac', '', 'Pro', 'iMac'],
 '1325': ['', 'Cable', 'to', 'USB', 'cable'],
 '5384': ['', 'iPhone', 'iPad', 'iPod', 'Bluetooth'],
 '1334': ['', 'Gigabit', 'ports', 'Wireless', 'Mac'],
 '13005399': ['', 'MacBook', 'MagSafe', 'Charger', 'Air'],
 '13835403': ['', 'MacBook', 'Pro', 'inches', '13-inch'],
 '12995397': ['', 'Mac', 'PC', 'Thunderbolt', 'USB'],
 '11865403': ['', 'iPhone', 'Plus', '6', 'case'],
 '13955395': ['', 'to', 'connector', 'digital', 'cameras'],
 '1216': ['', 'iPad', 'support', 'iPhone', 'Air'],
 '12355400': ['', 'USB', 'GoPro', 'monitors', 'color'],
 '5,49E+11': ['', 'iPhone', 'Case', 'battery', '6'],
 '1276': ['', 'Mac', 'card', 'PCIe', 'adapter'],
 '11905404': ['', 'iPhone', 'iPad', 'iPod', 'Touch'],
 '12635403': ['', 'iPad', 'Pro',

In [ ]:
type_dict['20642062']

['Installation of RAM in your MacBook / Pro.',
 'HDD or SSD installation on your MacBook / Pro + Data Transfer.',
 'RAM + HDD installation + SSD in your MacBook / Pro + Data Transfer.',
 'Installation of RAM in your iMac.',
 'RAM installed in your Mac mini.',
 'Installation of RAM in your Mac Pro.',
 'Installing SSD in your MacBook Pro Retina + Data Transfer.',
 'SSD installation on your MacBook Air + Data Transfer.',
 'HDD or SSD installation on your Mac Pro + Data Transfer.',
 'installation RAM HDD + SSD + on your Mac mini + Data Transfer.',
 'installation RAM HDD + SSD + on your iMac + Data Transfer.',
 'installation RAM HDD + SSD + on your iMac + Data Transfer.']

In [ ]:
string = ''
for value in type_dict['20642062']:
  string += ' '+ value
  list = string.split()
  new_list = []
  for l in list:
    l = l.replace('.','').replace('+', '/').replace('/','').replace('your', '')
    new_list.append(l)
  counter = Counter(new_list)
  most_occur = counter.most_common(5)
  words = []
  for i in range(5):
    words.append(most_occur[i][0])
words

['', 'RAM', 'SSD', 'Data', 'Transfer']

In [ ]:
'RAM + HDD installation + SSD in your MacBook / Pro + Data Transfer.'.replace('.','').replace('+', '').replace('/','')

'RAM  HDD installation  SSD in your MacBook  Pro  Data Transfer'

### Brands

,sku,name,desc,price,promo_price,in_stock,type
8698,APP2490,Apple iPhone 64GB X Silver,New Apple iPhone 64GB Free Silver X,115.900.092,11.590.009,1,113271716
